# $$
\begin{cases}
\Delta u =f \\
u \Big |_{\Gamma} = 0
\end{cases}
 \implies I(u) = \frac{1}{2}\iint_D\left[u_x^2 + u_y^2 \right]dxdy + \iint_Dfudxdy \rightarrow min$$

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Example
# $$ D = \left[-\frac{\pi}{2},+\frac{\pi}{2}\right]\times\left[-\frac{\pi}{2},+\frac{\pi}{2}\right] $$
# $$
\begin{cases}
\Delta u = 2 \\
u \Big |_{\partial D} = 0
\end{cases}
$$

In [ ]:
def f(x, y):
    return 2 * np.ones_like(x)

def g(x, y):
    return   np.zeros_like(x)

# Number of points per coordinate

In [ ]:
n_points = 20

# Construct the necessary matrices

In [ ]:
grid = np.linspace(-np.pi/2, np.pi/2, n_points)
grid_x, grid_y = np.meshgrid(grid, grid, indexing="ij")

In [ ]:
grid_z = f(grid_x, grid_y)
h_1 = h_2 = grid[1] - grid[0]

In [ ]:
b = -2 * grid_z.reshape(-1)
matrix = np.zeros((n_points**2, n_points**2), dtype=np.float32)

for i in range(n_points**2):
    idx = np.unravel_index(i, (n_points, n_points))
    
    #Dirichlet boundary conditions 
    if True:
        if idx[0] == 0:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue
        if idx[0] == n_points - 1:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue
        if idx[1] == 0:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue       
        if idx[1] == n_points - 1:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue

    matrix[i, i] = 4 * (1/h_1**2 + 1/h_2**2)
    
    if i + 1 < n_points**2:
        matrix[i, i+1] = -2/h_1**2
            
    if i - 1 > 0:
        matrix[i, i-1] = -2/h_1**2
            
    if i + n_points + 1 < n_points**2:
        matrix[i, i+n_points] = -2/h_2**2
            
    if i - n_points > 0:
        matrix[i, i-n_points] = -2/h_2**2

In [ ]:
u_simple = np.linalg.solve(matrix, b)
u_simple = u_simple.reshape(n_points, n_points)
u_simple += g(grid_x, grid_y)

In [ ]:
b = np.zeros(n_points**2)
matrix = np.zeros((n_points**2, n_points**2), dtype=np.float32)

for i in range(n_points**2):
    idx = np.unravel_index(i, (n_points, n_points))
    
    #Dirichlet boundary conditions 
    if True:
        if idx[0] == 0:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue
        if idx[0] == n_points - 1:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue
        if idx[1] == 0:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue       
        if idx[1] == n_points - 1:
            matrix[i] *= 0
            matrix[i, i] = 1
            b[i] = 0
            continue

    matrix[i, i] = 4 * (h_2/h_1 + h_1/h_2)
    
    if i + 1 < n_points**2:
        matrix[i, i+1] = -2*h_2/h_1
            
    if i - 1 > 0:
        matrix[i, i-1] = -2*h_2/h_1
            
    if i + n_points + 1 < n_points**2:
        matrix[i, i+n_points] = -2*h_1/h_2
            
    if i - n_points > 0:
        matrix[i, i-n_points] = -2*h_1/h_2
    
    
    edges = grid_z[idx[0]-1, idx[1]-1] + grid_z[idx[0]+1, idx[1]+1] +\
            grid_z[idx[0]-1, idx[1]+1] + grid_z[idx[0]+1, idx[1]-1]
    
    neigh = grid_z[idx[0]-1, idx[1]] + grid_z[idx[0]+1, idx[1]]+\
            grid_z[idx[0], idx[1]-1] + grid_z[idx[0], idx[1]+1]
    
    b[i] = -2 *(h_1*h_2)/36 * (edges + 4 * neigh + 16 * grid_z[idx])

# Solve the system of linear equations

In [ ]:
u_simpson = np.linalg.solve(matrix, b)
u_simpson = u_simpson.reshape(n_points, n_points)

In [ ]:
u_simpson += g(grid_x, grid_y)

# Basis function (pyramidal function)

In [ ]:
def basis(X, Y, h1, h2, i, j, n_x, n_y):
    tx = np.linspace(X.min(), X.max(), n_x)
    ty = np.linspace(Y.min(), Y.max(), n_y)
    xx, yy = np.meshgrid(tx, ty, indexing="ij")
    xm = xx.min()
    ym = yy.min()
    xx = xx - ym
    yy = yy - ym
    tmp = np.zeros(xx.shape)
    S1 = (i <= xx/h1) & (xx/h1 <= i + 1) & (j <= yy/h2) & (yy/h2 <= j + 1) & (yy/h2 - j >= xx/h1 - i)
    tmp[S1] = (j + 1 - yy/h2)[S1]
    S2 = (i - 1 <= xx/h1) & (xx/h1 <= i) & (j <= yy/h2) & (yy/h2 <= j + 1) & (yy/h2 - j <= 1 + xx/h1 - i)
    tmp[S2] = (1 + xx/h1 - yy/h2 - i + j)[S2]
    S3 = (i - 1 <= xx/h1) & (xx/h1 <= i) & (j - 1 <= yy/h2) & (yy/h2 <= j) & (yy/h2 - j >= xx/h1 - i)
    tmp[S3] = (1 + xx/h1 - i)[S3]
    S4 = (i - 1 <= xx/h1) & (xx/h1 <= i) & (j - 1 <= yy/h2) & (yy/h2 <= j) & (yy/h2 - j <= xx/h1 - i)
    tmp[S4] = (1 + yy/h2 - j)[S4]
    S5 = (i <= xx/h1) & (xx/h1 <= i + 1) & (j - 1 <= yy/h2) & (yy/h2 <= j) & (yy/h2 - j >= xx/h1 - i - 1)
    tmp[S5] = (1 - xx/h1 + yy/h2 + i - j)[S5]
    S6 = (i <= xx/h1) & (xx/h1 <= i + 1) & (j <= yy/h2) & (yy/h2 <= j + 1) & (yy/h2 - j <= xx/h1 - i)
    tmp[S6] = (1 - xx/h1 + i)[S6]
    return tmp

# Sum up all the basis functions

In [ ]:
t = np.linspace(grid[0], grid[-1], 200)
xx, yy, = np.meshgrid(t, t, indexing="ij")
z_simple = np.zeros_like(xx)
z_simpson = np.zeros_like(xx)

for i in range(n_points):
    for j in range(n_points):
        z_simple += basis(t, t, h_1, h_2, i, j, 200, 200)*u_simple[i][j]
        z_simpson += basis(t, t, h_1, h_2, i, j, 200, 200)*u_simpson[i][j]

# Visualisation

In [ ]:
%matplotlib widget
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax.plot_wireframe(xx, yy, z_simple, color="black")
# ax.plot_wireframe(xx, yy, z_simpson, color="red")
# ax.set_box_aspect((np.ptp(xx), np.ptp(yy), np.ptp(z_simpson)))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.imshow(matrix != 0)

# Error estimation

In [ ]:
z_original = np.zeros_like(xx)

In [ ]:
for k in range(1, 30):
    
    coef2 = (-1)**(k+1)/(2*k-1)**3
    ch1 = np.cosh((2*k-1)*yy)
    ch2 = np.cosh((2*k-1)*np.array(np.pi/2))
    cos = np.cos((2*k-1)*xx)
    
    z_original += (coef2*ch1/ch2*cos)

In [ ]:
z_original *= (8/np.pi)

In [ ]:
z_original += xx**2 - (np.pi / 2)**2

In [ ]:
error = (z_original-z_simple)
np.abs(error[~np.isnan(error)]).max().item()

In [ ]:
error = (z_original-z_simpson)
np.abs(error[~np.isnan(error)]).max().item()